## Importación de LightGlue

In [1]:
## En teoria ya esta en el repo, pero por si acaso

from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

fatal: destination path 'LightGlue' already exists and is not an empty directory.
/Users/manu/Proyecto Final/Proyecto-Final-Gondolieri/LightGlue


/Users/manu/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Modelo prueba

In [3]:
## Importo las librerias necesarias
! pip install numpy==1.26.4
import torch
from LightGlue.lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from LightGlue.lightglue.utils import load_image, rbd
from LightGlue.lightglue import viz2d
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import json
import os


In [4]:
# Defino parámetros de configuración
min_matches = 25  # Número mínimo de correspondencias necesarias para considerar que el objeto está presente
margin = 50  # Margen en píxeles alrededor del área recortada
overlap = 120  # Margen de superposición entre las partes

# Configurar el dispositivo y los modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
extractor = DoGHardNet(max_num_keypoints=2048).eval().to(device) #es donde se puede llamar a los modelos: DogHardNet, SuperPoint, SIFT, ALIKED, DISK
matcher = LightGlue(features="doghardnet").eval().to(device)

In [14]:
# Función para dividir la imagen de la góndola en 2 partes horizontales con superposición
def split_image_horizontal_with_overlap(image, overlap):
    h, w = image.shape[1], image.shape[2]
    split_height = h // 2

    upper_split = image[:, :split_height + overlap, :]
    lower_split = image[:, max(0, split_height - overlap):, :]

    return upper_split, lower_split

# Cargar la imagen de la góndola
gondola_image_path = "/Users/manu/Downloads/Mayo-Junio-Julio (teoricamente) 2/39-MJJ-.jpeg"
gondola_image = load_image(gondola_image_path)

gondola_upper, gondola_lower = split_image_horizontal_with_overlap(gondola_image, overlap)

# Ruta a la carpeta de productos
product_folder = Path("/Users/manu/Downloads/Cuidado Capilar/Tresseme")

# Procesar cada imagen de producto
for product_image_path in product_folder.glob('*'):  
    if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']: 
        print(f"Analizando el producto: {product_image_path.name}")

        product_image = load_image(product_image_path)
        # Procesar cada partición de la imagen de la góndola
        for idx, gondola_split in enumerate([gondola_upper, gondola_lower]):
            feats0 = extractor.extract(product_image.to(device))
            feats1 = extractor.extract(gondola_split.to(device))
            matches01 = matcher({"image0": feats0, "image1": feats1})
            feats0, feats1, matches01 = [
                rbd(x) for x in [feats0, feats1, matches01]
            ] 

            kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
            m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

            # Visualizar los resultados
            axes = viz2d.plot_images([product_image, gondola_split])
            viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
            viz2d.add_text(0, f'Stop after {matches01["stop"]} layers in split {idx}', fs=20)


            # Verificar si el número de correspondencias es suficiente
            if len(matches) >= min_matches:
                print(f"Objeto detectado con {len(matches)} correspondencias en la partición {idx} para el producto: {product_image_path.name}")

                # Cargar las bounding boxes del archivo JSON
                json_path = gondola_image_path.replace(Path(gondola_image_path).suffix, '.json')
                with open(json_path, 'r') as f:
                    bounding_boxes = json.load(f)

                # Verificar si el producto tiene una bounding box en el JSON
                product_name = product_image_path.stem
                if product_name in bounding_boxes:
                    bbox = bounding_boxes[product_name]
                    x_min, y_min, x_max, y_max = bbox

                    # Contar cuántos puntos de coincidencia están dentro de la bounding box
                    in_bbox_count = sum(
                        1 for x, y in m_kpts1 if x_min <= x <= x_max and y_min <= y <= y_max
                    )

                    # Verificar si la mayoría de los puntos de coincidencia están dentro de la bounding box
                    if in_bbox_count / len(m_kpts1) > 0.5:
                        print(f"Producto {product_name} detectado correctamente en la partición {idx} con {in_bbox_count} puntos dentro de la bounding box.")
                    else:
                        print(f"Falso positivo para el producto {product_name} en la partición {idx}. Solo {in_bbox_count} puntos dentro de la bounding box.")
                else:
                    print(f"Falso positivo para el producto {product_name} en la partición {idx}. No se encontró bounding box en el JSON.")
            else:
                print(f"Objeto no detectado o no confiable, solo {len(matches)} correspondencias encontradas en la partición {idx} para el producto: {product_image_path.name}")

        print(f"Análisis completado para el producto: {product_image_path.name}")



Analizando el producto: TRESEMME SH LISO BOTOX ST P 12X500ML - 7791293050119.png


RuntimeError: Numpy is not available